# Clustering - Tuto 0

In this 1st tuto, we simply play with KMeans and GMM on a toy multimodal dataset

The objective is to illustrate a "clustering" method

In [ ]:
# Libraries import section
import os

import xarray as xr
import numpy as np
from scipy import stats
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

import seaborn as sns
sns.set(context="notebook", style="whitegrid", palette="deep", color_codes=True)

In [ ]:
# Usefull functions

# Trivial functions to manipulate 2D matrices [n_samples,n_features=2]

def stretch(X,m=[1,1]):
    """Stretch a 2d matrix X by factors m[0] and m[1] along dimemsions 0 and 1"""
    return np.dot(X, [[m[0], 0], [0, m[1]]])

def rotate(X,a=0):
    """Rotate around the origin a 2d matrix X by angle a in degrees"""
    a = np.deg2rad(a) # Convert degrees to radians
    return np.dot(X, [[np.cos(a), np.sin(a)], [-np.sin(a), np.cos(a)]])

def translate(X,v=[0,0]):
    """Translate 2d matrix X by v[0] and v[1] along dimemsions 0 and 1"""
    n = X.shape[0]
    X[:,0] += np.full((n),v[0])
    X[:,1] += np.full((n),v[1])
    return X

def new_sample(m,s,n=100,r=170):
    """Return a 2D normal distribution centered on m with std s"""
    X, y = make_blobs(n_samples=n, random_state=r, centers=[m], cluster_std=s)
    return {'data':X,'labels':y}


# Plotting functions

def plot2d_labels(X,labels,cluster_centers):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=plt.figaspect(0.6), dpi=90, facecolor='w', edgecolor='k')

    unique_labels = np.unique(labels)
    n_clusters_ = unique_labels.shape[0]
    colors = sns.husl_palette(n_clusters_)

    for k, col in zip(range(n_clusters_), colors):
        class_members = labels == k

        plt.plot(X[class_members,0], X[class_members,1], '.', 
                 markerfacecolor=tuple(col), markeredgecolor='none', label='Class %i'%(k))

        plt.plot(cluster_centers[0], cluster_centers[1], 'p', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=10, label='Class center #%i'%(k))

    # plt.xlim(-3,9)
    # plt.ylim(-3,3)
    plt.xlabel("Feature 0")
    plt.ylabel("Feature 1")
    plt.title('Number of clusters: %i' % n_clusters_)
    plt.legend()
    plt.show()

def plot_GMMellipse(gmm,id,ik,col,ax,label="",std=[1],main_axes=True,**kwargs):
    """
        Plot an 1-STD ellipse for a given component (ik) and 2 given dimensions (id) 
        of the GMM model gmm
        This is my routine, simply working with a matplotlib plot method
        I also added the possiblity to plot the main axes of the ellipse
    """
    covariances = gmm.covariances_[ik][(id[0],id[0],id[1],id[1]),(id[0],id[1],id[0],id[1])].reshape(2,2)
    d, v = np.linalg.eigh(covariances) #  eigenvectors have unit length
    d = np.diag(d)
    theta = np.arange(0,2*np.pi,0.02)
    x = np.sqrt(d[0,0])*np.cos(theta)
    y = np.sqrt(d[1,1])*np.sin(theta)
    xy = np.array((x,y)).T
    ii = 0
    for nstd in np.array(std):
        ii+=1
        ellipse = np.inner(v,xy).T
        ellipse = nstd*ellipse + np.ones((theta.shape[0], 1))*gmm.means_[ik,(id[0],id[1])]
        if ii == 1:
    #            p = ax.plot(ellipse[:,0], ellipse[:,1], color=col, axes=ax, label=("%s (%i-std)")%(label,nstd),**kwargs)
            p = ax.plot(ellipse[:,0], ellipse[:,1], color=col, axes=ax, label=("%s")%(label),**kwargs)
        else:
            p = ax.plot(ellipse[:,0], ellipse[:,1], color=col, axes=ax,**kwargs)
    if main_axes: # Add Main axes:
        for idir in range(2):
            l = np.sqrt(d[idir,idir])*v[:,idir].T
            start = gmm.means_[ik,(id[0],id[1])]-l
            endpt = gmm.means_[ik,(id[0],id[1])]+l
            linex = [start[0], endpt[0]]
            liney = [start[1], endpt[1]]
            plt.plot(linex,liney,color=col,axes=ax)
    return p,ax

In [ ]:
# Create a 2D sample datasets with Normal distributions
# Dataset is in "X", true labels in "y"

n = 100 # Number of samples per cluster

il = 0 # Init cluster IDs
s = new_sample([10,0],1,n=n)
X, y = s['data'], s['labels'] # 1st label is "0"

il += 1
X, y = np.concatenate((X,new_sample([5,0],2,n=n)['data']),axis=0), np.concatenate((y,np.full((n),il)),axis=0)

il += 1
X, y = np.concatenate((X,new_sample([5,5],0.7,n=n)['data']),axis=0), np.concatenate((y,np.full((n),il)),axis=0)

il += 1
x = translate(rotate(stretch(new_sample([0,0],1,n=2*n)['data'],m=[5,1]),a=45),v=[0,5])
X, y = np.concatenate((X,x),axis=0), np.concatenate((y,np.full((2*n),il)),axis=0)

n_clusters_true = il+1

plt.figure(figsize=(10,10), dpi=160, facecolor='w', edgecolor='k')
plt.subplot(221)
plt.plot(X[:, 0], X[:, 1],'k.',markersize=5)
plt.axis('equal')
plt.title("Raw dataset")
plt.subplot(222)
plt.scatter(X[:, 0], X[:, 1], 5, c=y, cmap=ListedColormap(sns.husl_palette(n_clusters_true).as_hex()))
plt.axis('equal')
plt.title("True %i clusters"%n_clusters_true)

In [ ]:
# Clustering with KMeans
kmeans = KMeans(n_clusters=n_clusters_true, random_state=0)
kmeans.fit(X)
labels = kmeans.predict(X)
n_clusters_ = kmeans.n_clusters
kmeans.cluster_centers_.shape

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
plot2d_labels(X,labels,cluster_centers=kmeans.cluster_centers_)

In [ ]:
# Clustering with GMM
gmm = GMM(n_components=n_clusters_true, random_state=0).fit(X)
labels = gmm.predict(X)
n_clusters_ = gmm.n_components
# print gmm.means_
# print gmm.covariances_

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
plot2d_labels(X,labels,cluster_centers=gmm.means_)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=plt.figaspect(0.6), dpi=90, facecolor='w', edgecolor='k')

unique_labels = np.unique(labels)
n_clusters_ = unique_labels.shape[0]
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for k, col in zip(range(n_clusters_), colors):
#     print "Cluster: ", k, col
    class_members = labels == k
    plt.plot(X[class_members,0], X[class_members,1], '.', 
             markerfacecolor=tuple(col), markeredgecolor='none', label='Class %i'%(k))

for k, col in zip(range(n_clusters_), colors):
#     cluster_center = gmm.means_[k,:]
#     plt.plot(cluster_center[0], cluster_center[1], 'p', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=10, label='Class center #%i'%(k))
    el,ax = plot_GMMellipse(gmm,[0,1],k,'k',ax,label="Class-%i"%(k),linewidth=2)

ax.axis('equal')
plt.title('Number of clusters: %i (GMM)' % n_clusters_)
plt.legend()
plt.show()